In [2]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'matplotlib'

In [6]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


NameError: name 'cifar10' is not defined

In [ ]:
# Normalize the dataset
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
# Display some sample images
for i in range(5):
    plt.imshow(x_train[i])
    plt.title(f"Label: {np.argmax(y_train[i])}")
    plt.show()

In [ ]:
def create_model(activation='relu', pooling='max'):
    model = models.Sequential()
    
    # Add convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation=activation, input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation=activation))
    
    # Add pooling layer
    if pooling == 'max':
        model.add(layers.MaxPooling2D((2, 2)))
    elif pooling == 'avg':
        model.add(layers.AveragePooling2D((2, 2)))
    
    # Add dropout
    model.add(layers.Dropout(0.25))
    
    # Flatten and add dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
def experiment_with_models(activations, poolings, num_epochs=10, batch_size=64):
    results = []
    
    for activation in activations:
        for pooling in poolings:
            print(f"Training model with Activation: {activation}, Pooling: {pooling}")
            
            # Create and train the model
            model = create_model(activation=activation, pooling=pooling)
            history = model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size,
                                validation_data=(x_test, y_test), verbose=1)
            
            # Evaluate the model
            test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
            print(f"Test Accuracy: {test_acc:.4f}")
            
            # Store the results
            results.append({
                'activation': activation,
                'pooling': pooling,
                'test_accuracy': test_acc,
                'history': history.history
            })
    
    return results


In [ ]:
# Function to plot training vs. validation accuracy
def plot_results(results):
    for result in results:
        history = result['history']
        activation = result['activation']
        pooling = result['pooling']
        
        # Plot accuracy
        plt.figure(figsize=(10, 6))
        plt.plot(history['accuracy'], label='Training Accuracy')
        plt.plot(history['val_accuracy'], label='Validation Accuracy')
        plt.title(f"Activation: {activation}, Pooling: {pooling}")
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid()
        plt.show()

# Function to summarize results
def summarize_results(results):
    summary = pd.DataFrame([{
        'Activation': result['activation'],
        'Pooling': result['pooling'],
        'Test Accuracy': result['test_accuracy']
    } for result in results])
    
    print(summary.sort_values(by='Test Accuracy', ascending=False))


In [ ]:
# Define the configurations to test
activations = ['relu', 'tanh', 'sigmoid']
poolings = ['max', 'avg']

# Run experiments
results = experiment_with_models(activations, poolings, num_epochs=5)

# Visualize the results
plot_results(results)

# Summarize the results
summarize_results(results)
